In [23]:
import sys
sys.path.append('./../../src')

# python's shit
from os import path
from datetime import date, datetime, timedelta
import time
import calendar

# someone elses shit
import finnhub
import pandas

# my shit
import config
from lib.stonk_jar import StonkJar

In [2]:
finnhub_client = finnhub.Client(api_key=config.api_keys['finnhub']['sandbox'])
ticker = 'AMC'
jar = StonkJar(ticker)

In [3]:
# Time junk, just for testing?
now = time.time()
today = date.today()
yesterday = today + timedelta(days = -1)

In [13]:
def recommendation_trends_by_date(ticker, date):
    first_of_month = date.replace(day = 1)
    recommendations = jar.pickle_back(
        "{0}_recommendations.pkl".format(ticker),
        finnhub_client.recommendation_trends,
        ticker)
    recommendation = [r for r in recommendations if datetime.strptime(r['period'], '%Y-%m-%d').date() == first_of_month]
    if (len(recommendation) > 0):
        return recommendation[0]
    return {}

In [12]:
def company_earnings_by_date(ticker, date):
    reporting_period = timedelta(days = 90)
    earnings = jar.pickle_back(
        "{0}_earnings.pkl".format(ticker),
        finnhub_client.company_earnings,
        ticker)
    earning = [r for r in earnings if datetime.strptime(r['period'], '%Y-%m-%d').date() + reporting_period > date]
    if (len(earning) > 0):
        return earning[0]
    return {}

Build a base data frame from the stock's low, open, close, high and volume for the given time period

In [6]:
def stock_candles_by_date(ticker, date):
    first_of_month_ts = int(datetime.combine(date.replace(day = 1), datetime.min.time()).timestamp())
    next_month = date.replace(day = 28) + timedelta(days = 4)
    last_of_month = next_month - timedelta(days = next_month.day)
    last_of_month_ts = datetime.combine(last_of_month, datetime.max.time()).timestamp()
    earliest = int(min(time.time(), last_of_month_ts))
    candles = finnhub_client.stock_candles('AAPL', '60', first_of_month_ts, earliest)
    return zip(candles['l'], candles['o'], candles['c'], candles['h'], candles['v'])

Construct a set of data frames that each hold:
- low
- open
- close
- high
- volume
- eps_actual (the actual EPS of the last relevant reporting period of the data frame's time period)
- eps_estimate (the estimated EPS, same as above)
- rec_strong_sell
- rec_sell
- rec_hold
- rec_buy
- rec_strong_sell (this and the above 4 fields are proportions of each rec cat of the total [0 - 1])

In [19]:
def construct_data_frames(ticker, date):
    trend = recommendation_trends_by_date(ticker, date)
    trend_total = trend['strongSell'] + trend['sell'] + trend['hold'] + trend['buy'] + trend['strongBuy']
    earnings = company_earnings_by_date(ticker, date)
    static_data = []
    if 'actual' in earnings and 'estimate' in earnings:
        static_data = static_data.append(earnings['actual'], earnings['estimate'])
    static_data = static_data + [
        trend['strongSell'] / trend_total, trend['sell'] / trend_total,
        trend['hold'] / trend_total,
        trend['buy'] / trend_total, trend['strongBuy'] / trend_total
    ]
    base_frames = stock_candles_by_date(ticker, date)
    return [list(x) + static_data for x in base_frames]

Go through the calendar and fetch all the historical data on this ticker that we have access to.
Let's start with 3 months.

In [8]:
def get_historical_data(ticker, days = 90):
    # if this historical pickle file exists, just return it
    historical_pickle_name = "{0}.technical.historical.df.pkl".format(ticker)
    if jar.pickle_exists(historical_pickle_name):
        return jar.read_pickle_dataframe(historical_pickle_name)
    # if it doesn't, build it
    historical_data = pandas.DataFrame()
    today = date.today()
    x_days_ago = today + timedelta(days = -1 * days)
    current_date = x_days_ago
    while current_date < today:
        # look for pickle file for this days data for this day's ticker
        pickle_name = "{0}-{1}.technical.df.pkl".format(ticker, current_date.strftime("%m-%d-%Y"))
        if jar.pickle_exists(pickle_name):
            data = jar.read_pickle_dataframe(pickle_name)
        else:
            data = construct_data_frames(ticker, current_date)
            # pickle this day's data to cut down on API requests
            df = pandas.DataFrame.from_records(data)
            jar.write_pickle_dataframe(pickle_name, df)
            time.sleep(2) # sleep for 2 seconds so we don't hit the API limit
        historical_data = historical_data.append(data)
        current_date = current_date + timedelta(days = 1)
    # pickle this historical data
    jar.write_pickle_dataframe(historical_pickle_name, historical_data)
    return historical_data

In [20]:
df = get_historical_data(ticker)

In [50]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11
count,63316.000000,63316.000000,63316.000000,63316.000000,63316.000000,63316.000000,63316.000000,63316.000000,63316.000000,63316.000000,23716.0,23716.0
mean,195.631298,196.219957,196.297010,197.004837,49.762967,-3.119571,-1.798022,0.341641,0.255305,0.297910,0.0,0.0
std,52.110836,52.267639,52.289833,52.476709,28.588117,4.094438,2.480081,0.191928,0.160502,0.244991,0.0,0.0
min,120.741238,121.104551,121.165225,121.588970,1.000000,-8.410000,-4.999500,0.116883,0.000000,0.000000,0.0,0.0
25%,150.998355,151.452713,151.516912,152.058524,25.000000,-8.410000,-4.999500,0.116883,0.000000,0.000000,0.0,0.0
50%,187.973783,188.539402,188.581433,189.293559,50.000000,0.000000,0.000000,0.389381,0.285714,0.203540,0.0,0.0
75%,233.471665,234.174187,234.283435,235.110884,75.000000,0.116883,0.285714,0.597403,0.407080,0.597403,0.0,0.0
max,362.305366,363.395552,364.231362,364.849134,99.000000,0.116883,0.285714,0.597403,0.407080,0.597403,0.0,0.0


Now that we have raw hr resolution tick data, split it up into data 'complete' frames for training/validating. Each frame will have 10 ticks of data, 9 training ticks and the last verification ticks. This set of data will be randomized, split into a training set and a validation set, and then used to train the network.

In [60]:
real_frames = []
x = 0; total = len(df)
while x < total:
    real_frames = real_frames + [df.take(range(x, min(total-1, x+10)))]
    x += 1

In [62]:
jar.write_pickle_file("{0}-all-data-{1}.pkl".format(ticker, date.today().strftime('%Y-%m-%d')), real_frames)

AttributeError: 'StonkJar' object has no attribute 'to_pickle_file'